In [1]:
from typing import Annotated

from typing_extensions import TypedDict
from langgraph.graph import START , END , StateGraph
from langgraph.graph.message import add_messages


In [2]:
class State(TypedDict):
    # messages have the type "list" . add_messages function in 
    # the anotation defines how the state key should be updated 
    # in this case it appends the list instead of overwriting
    messages : Annotated[list[str] , add_messages]
    

In [3]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm = ChatGroq(model = "llama3-8b-8192")

# you have to initilize the model name 

In [5]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021C36A7CEC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021C36A7DA90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
def chatbot(state : State):
    return {"messages":[llm.invoke(state["messages"])]}

In [7]:
graph_builder = StateGraph(State)

# adding the node 
graph_builder.add_node("llmchatbot" , chatbot)
# adding the edges 
graph_builder.add_edge(START , "llmchatbot")
graph_builder.add_edge("llmchatbot" , END)

# compile the graph 
graph = graph_builder.compile()

In [8]:
response = graph.invoke({"messages" : "Hi"})

# this message will be appeneded in the State defined using annotated 


In [9]:
response['messages'][-1].content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [10]:
for event in graph.stream({"messages" : "hi How are you?"}):
    for ev in event.values():
        print(ev["messages"][-1].content)
    
# it shows that whenever you do the graphstream it will only give you the ai messgaes not the humans 



Hi! I'm just an AI, so I don't have feelings like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going?


### chatbot using Tools Tavily 

In [11]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_result = 2)
tool.invoke("who current prime minister of india")

{'query': 'who current prime minister of india',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Prime_Minister_of_India',
   'title': 'Prime Minister of India - Wikipedia',
   'content': 'Modi is the current prime minister of India, serving since 26 May 2014 and the first to win three consecutive elections to secure a third successive term, 2014,',
   'score': 0.9016528,
   'raw_content': None},
  {'url': 'https://www.pmindia.gov.in/en/pms-profile/',
   'title': 'Know the PM | Prime Minister of India',
   'content': "Shri Narendra Modi was sworn-in as India's Prime Minister for the third time on 9th June 2024, following another decisive victory in the 2024 Parliamentary",
   'score': 0.84858394,
   'raw_content': None},
  {'url': 'http://www.pmindia.gov.in/en/',
   'title': 'Prime Minister of India',
   'content': "Shri Narendra Modi was sworn-in as India's Prime Minister for the third time on 9th June 2024, following a

In [12]:
def multiply(a:int , b:int)->int:
    """
    
    """
    return a*b

In [13]:
tools = [tool , multiply]

In [14]:
llmwithtools = llm.bind_tools(tools)

# it will bind llm with tools 

In [15]:
llmwithtools

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021C36A7CEC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021C36A7DA90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets, but offers advanced search depths, domain management, time range filters, and image search, this tool delivers real-time, accurate, and citation-backed results.Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'Search query to look up', 'type': 'string'}, 'include_domains': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'default': [], 'description': 'A list 

In [16]:
from langgraph.graph import START ,END , StateGraph
from langgraph.prebuilt import ToolNode ,tools_condition
# it is used for cheking which tool is used by llm 

from langgraph.checkpoint.memory import MemorySaver


memory = MemorySaver()

def tool_calling_llm(state:State):
    return {"messages":[llmwithtools.invoke(state['messages'])]}

# graph 
builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools" , ToolNode(tools))

builder.add_edge(START , "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # if the latest message from assistant is tool call -> tool conditon routes to tools 
    # if the latest message from assistant is not a tool call -> tool condition routes to END 
    # that is what tools_condition does 
    tools_condition
)

builder.add_edge("tools" , END)    
# you can make it react agent architecture as well   END->tool_calling_llm

graph = builder.compile()

In [17]:
response = graph.invoke({"messages" : "give cpp code for prime number"})

# here you caan see , there is no ai message , it does a tool call and you can see the tool message 

In [18]:
for n in response['messages']:
    n.pretty_print()
    
# here you can see the llm made multiply tool call 

================================ Human Message =================================

give cpp code for prime number
================================== Ai Message ==================================

Here is an example of C++ code that checks if a number is prime:
```
#include <iostream>

bool isPrime(int n) {
  if (n <= 1) return false; // 0 and 1 are not prime
  for (int i = 2; i * i <= n; i++) {
    if (n % i == 0) return false; // if n is divisible by i, it's not prime
  }
  return true; // if n is not divisible by any i, it's prime
}

int main() {
  int num;
  std::cout << "Enter a number: ";
  std::cin >> num;
  if (isPrime(num)) {
    std::cout << num << " is a prime number." << std::endl;
  } else {
    std::cout << num << " is not a prime number." << std::endl;
  }
  return 0;
}
```
This code uses a simple algorithm to check if a number is prime. It works by iterating from 2 to the square root of the number and checking if the number is divisible by any of the values in that range.

In [19]:
response = graph.invoke({"messages" : "multiply 2178 by 212 and also give me the recent ai news"})


In [20]:
for n in response['messages']:
    n.pretty_print()
    
# bith of the tools calls are made 


# there is another concept of react agent arcchitecture in which after 
# you tool call it doenot ends , it again go to llm, and perfrom the cycle
# untill the answer is totally satisfied.



================================ Human Message =================================

multiply 2178 by 212 and also give me the recent ai news
================================== Ai Message ==================================
Tool Calls:
  multiply (2sbmybv4t)
 Call ID: 2sbmybv4t
  Args:
    a: 2178
    b: 212
  tavily_search (mey2pt6w6)
 Call ID: mey2pt6w6
  Args:
    query: recent ai news
    search_depth: advanced
    topic: general
================================= Tool Message =================================
Name: multiply

461736
================================= Tool Message =================================
Name: tavily_search

{"query": "recent ai news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.crescendo.ai/news/latest-ai-news-and-updates", "title": "Latest AI Breakthroughs and News: May, June, July 2025", "content": "New AI Model Predicts Human Decisions with Surprising Precision . Summary: Scientists have developed an AI model t

### Adding memory for a session of Chatbot

In [21]:
# here we use memory saver funciton of langgraph to save the previous interaction 
# and we have to add this memory while compiling a graaph

from langgraph.graph import START ,END , StateGraph
from langgraph.prebuilt import ToolNode ,tools_condition
# it is used for cheking which tool is used by llm 

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
# memory saver function 

def tool_calling_llm(state:State):
    return {"messages":[llmwithtools.invoke(state['messages'])]}

# graph 
builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools" , ToolNode(tools))

builder.add_edge(START , "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # if the latest message from assistant is tool call -> tool conditon routes to tools 
    # if the latest message from assistant is not a tool call -> tool condition routes to END 
    # that is what tools_condition does 
    tools_condition
)

builder.add_edge("tools" , END)    
# you can make it react agent architecture as well   END->tool_calling_llm

graph = builder.compile(checkpointer=memory)

In [22]:
# we have to first configure the session by giving the thread id 
# so that the model should store the data in memory for that perticular session with id 

config = {"configurable":{"thread_id" : "1"}}  
# thread id should be unique 

# again invoke the graph 
response = graph.invoke({"messages" : "hi my name is shantanu"} , config=config)

# we have to also tell the model for which thread we are providing the message, i.e. ->  config = config
response['messages'][-1].content


'Nice to meet you, Shantanu! How can I assist you today?'

In [23]:
response = graph.invoke({"messages" : "hi what is my good name ? and how do you remember my name ?"} , config=config)

# now it will give my name because previously i have given a prompt of my name , and model saved it in this session
response['messages'][-1].content

'I\'ve been trained on various natural language processing techniques, including machine learning algorithms, to recognize and remember names. When you introduced yourself as "Shantanu", I stored that information in my memory so that I can recall it later.\n\nAs for your good name, I think it\'s a lovely question! In many cultures, people are referred to by various names, such as a given name, surname, nickname, or even a title. Could you please elaborate on what you mean by "good name"? Are you referring to a specific cultural or spiritual context, or is it a personal question?\n\nFeel free to share more, and I\'ll do my best to understand and respond thoughtfully!'

### Streaming in Langgraph 

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory= MemorySaver()

In [25]:
def superbot(state : State):
    return {"messages" : [llm.invoke(state['messages'])]}

In [26]:
# graph 
graph = StateGraph(State)
graph.add_node("superbot" , superbot)
graph.add_edge(START , "superbot")
graph.add_edge("superbot" , END)

graph_builder = graph.compile(checkpointer=memory)

In [27]:
#there are some streaming methods called as stream , astream.
# addtional parameters are values and updates 
# values = You get only the final output(s) of the graph — i.e., the end result(s) of your flow 
# updates = You get detailed events — each time a node runs, edges are traversed, and state is updated.


config = {"configurable":{"thread_id" : "1"}}  
# thread id should be unique 

# again invoke the graph 
for chunk in graph_builder.stream({"messages" : "hi my name is shantanu , i like football"} , config , stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='hi my name is shantanu , i like football', additional_kwargs={}, response_metadata={}, id='2fe4bcf4-6cb5-4b32-84b4-510c2a681cc4')]}
{'messages': [HumanMessage(content='hi my name is shantanu , i like football', additional_kwargs={}, response_metadata={}, id='2fe4bcf4-6cb5-4b32-84b4-510c2a681cc4'), AIMessage(content='Hi Shantanu! Nice to meet you! Football, eh? Which team do you support?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 21, 'total_tokens': 42, 'completion_time': 0.014808489, 'prompt_time': 0.00304615, 'queue_time': 0.287159621, 'total_time': 0.017854639}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8b7c3a83f7', 'finish_reason': 'stop', 'logprobs': None}, id='run--110aab90-f825-439a-b5b1-29832642ba70-0', usage_metadata={'input_tokens': 21, 'output_tokens': 21, 'total_tokens': 42})]}


In [28]:
config = {"configurable":{"thread_id" : "2"}}  


for chunk in graph_builder.stream({"messages" : "hi my name is shantanu , i also like tennis"} , config , stream_mode="updates"):
    print(chunk)

{'superbot': {'messages': [AIMessage(content="Hello Shantanu! Nice to meet you! Tennis is a great sport, isn't it? Who's your favorite tennis player? Roger Federer, Rafael Nadal, or maybe Novak Djokovic?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 22, 'total_tokens': 66, 'completion_time': 0.069668569, 'prompt_time': 0.005328489, 'queue_time': 0.28183841, 'total_time': 0.074997058}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8dc6ecaf8e', 'finish_reason': 'stop', 'logprobs': None}, id='run--324ca43b-e2a1-4869-8174-8adcbc829fa9-0', usage_metadata={'input_tokens': 22, 'output_tokens': 44, 'total_tokens': 66})]}}
